# Slalom turns

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot


pyplot.style.use('seaborn')

In [ ]:
from dataclasses import dataclass

from numpy import cos
from numpy import pi
from numpy import sin
from numpy import sign
from numpy import sqrt
from pandas import DataFrame


@dataclass
class RobotPhysics:
    mass: float
    moment_of_inertia: float
    wheels_separation: float
    max_angular_velocity: float


def turn_profile(angle, max_angular_velocity,
                 max_angular_acceleration, time_period):
    duration = max_angular_velocity / max_angular_acceleration * pi
    transition_angle = duration * max_angular_velocity / pi
    arc = (abs(angle) - 2 * transition_angle) / max_angular_velocity
    transition = duration / 2
    max_angular_velocity = max_angular_velocity * sign(angle)

    angular_velocity = [0]
    time = [0]
    while True:
        if time[-1] >= 2 * transition + arc:
            break
        new_angular_velocity = max_angular_velocity
        if time[-1] < transition:
            factor = time[-1] / transition
            new_angular_velocity *= sin(factor * pi / 2)
        elif time[-1] >= transition + arc:
            factor = (time[-1] - arc) / transition
            new_angular_velocity *= sin(factor * pi / 2)
        angular_velocity.append(new_angular_velocity)
        time.append(time[-1] + time_period)

    profile = DataFrame({'angular_velocity': angular_velocity[1:]})
    profile['arc'] = profile['angular_velocity'] == max_angular_velocity
    return profile


def complete_profile(profile, radius, linear_velocity, robot, time_period):
    profile['radius'] = radius
    profile['period'] = time_period
    profile['linear_velocity'] = linear_velocity
    profile['angle'] = (profile['angular_velocity'] * profile['period']).cumsum()
    profile['x'] = (profile['linear_velocity'] * cos(profile['angle']) * profile['period']).cumsum()
    profile['y'] = (profile['linear_velocity'] * sin(profile['angle']) * profile['period']).cumsum()
    profile['centrifugal_force'] = robot.mass * profile['linear_velocity'] * profile['angular_velocity'] / 2
    angular_acceleration = profile['angular_velocity'].diff() / profile['period']
    profile['angular_acceleration_force'] = \
        (robot.moment_of_inertia * angular_acceleration / robot.wheels_separation).abs()
    profile['total_force'] = sqrt(profile['centrifugal_force'] ** 2 + profile['angular_acceleration_force'] ** 2)
    profile['time'] = profile['period'].cumsum()
    profile = profile.set_index('time')
    return profile


def dynamic_profile(angle, force, radius, robot, time_period):
    linear_velocity = sqrt(2 * force * radius / robot.mass)
    max_angular_velocity = linear_velocity / radius
    max_angular_velocity = min(max_angular_velocity,
                               robot.max_angular_velocity)
    max_angular_acceleration = force * robot.wheels_separation / robot.moment_of_inertia
    max_angular_velocity_transition = sqrt(abs(angle) / 2 * max_angular_acceleration)
    if max_angular_velocity_transition < max_angular_velocity:
        raise ValueError
    profile = turn_profile(angle, max_angular_velocity,
                           max_angular_acceleration, time_period)
    return complete_profile(profile, radius, linear_velocity, robot, time_period)


def static_profile(angle, force, robot, time_period):
    angular_acceleration = force * robot.wheels_separation / robot.moment_of_inertia
    max_angular_velocity = sqrt(abs(angle) / 2 * angular_acceleration)
    max_angular_velocity = min(max_angular_velocity,
                               robot.max_angular_velocity)
    profile = turn_profile(angle, max_angular_velocity,
                           angular_acceleration, time_period)
    return complete_profile(profile, 0., 0., robot, time_period)


def describe(profile):
    print('Final position: (%.5f, %.5f)' % (profile['x'].iloc[-1], profile['y'].iloc[-1]))
    angular = profile['angular_velocity']
    step = profile['linear_velocity'].iloc[0] * profile['period'].iloc[0]
    arc = profile['arc'].sum() * step
    transition = (~profile['arc']).sum() * step / 2
    print('{xxx, xxx, %.5f, %.5f, %.5f, xxx}' % (profile['radius'].iloc[0], transition, arc))


def plot_profile(profile):
    profile[['centrifugal_force', 'angular_acceleration_force', 'total_force']].plot(style='.-')
    profile[['angular_velocity']].plot(style='.-')
    profile[['angle']].plot(style='.-')
    pyplot.figure(figsize=(6, 6))
    pyplot.plot(profile['x'], profile['y'], '.-')
    pyplot.show()

In [ ]:
time_period = 0.00001

bulebule = RobotPhysics(
    mass=0.110,
    moment_of_inertia=(0.110 * 0.05 ** 2) / 2,
    wheels_separation=0.065,
    max_angular_velocity=20,
)

## Static turns

In [ ]:
max_angular_velocity = 10.
profile = static_profile(
    angle=pi,
    force=0.25,
    robot=bulebule,
    time_period=time_period)
describe(profile)

## Search turn

In [ ]:
profile = dynamic_profile(
    angle=pi/2,
    force=0.25,
    radius=0.04921,
    robot=bulebule,
    time_period=time_period,
)
describe(profile)

## 90º turn

In [ ]:
profile = dynamic_profile(
    angle=pi/2,
    force=0.25,
    radius=0.1172,
    robot=bulebule,
    time_period=time_period,
)
describe(profile)

## 180º turn

## Turn to 45º

## Turn to 135º

## Turn diagonal 90º